<a href="https://colab.research.google.com/github/everest8849top/SMS-Text-Classification/blob/main/fcc_sms_text_classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.16.0-dev20231013


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-10-16 17:39:49--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2023-10-16 17:39:49 (10.5 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2023-10-16 17:39:49--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

In [ ]:
train_set = pd.read_csv(train_file_path, sep = '\t', names = ["class", "message"])
test_set = pd.read_csv(test_file_path, sep='\t', names=["class", "message"])

In [ ]:
train_message = train_set["message"].values.tolist()
train_label = np.array([0 if x=="ham" else 1 for x in train_set['class'].values.tolist()])
test_message = test_set["message"].values.tolist()
test_label = np.array([0 if x=="ham" else 1 for x in test_set['class'].values.tolist()])

In [ ]:
vocabulary_dict = {}
for messgae in train_message:
  for vocabulary in messgae.split():
    if vocabulary not in vocabulary_dict:
      vocabulary_dict[vocabulary] = 1
    else:
      vocabulary_dict[vocabulary] += 1

In [ ]:
VOCAB_SIZE = len(vocabulary_dict)
MAX_LENGTH = len(max(train_message, key=lambda p: len(p.split())).split())

In [ ]:
from tf.keras.preprocessing.text import one_hot
from tf.keras.preprocessing.sequence import pad_sequences

encoded_train_message = [one_hot(d, VOCAB_SIZE) for d in train_message]
padded_train_message = pad_sequences(encoded_train_message, maxlen=MAX_LENGTH, padding='post')
encoded_test_message = [one_hot(d, VOCAB_SIZE) for d in test_message]
padded_test_message = pad_sequences(encoded_test_message, maxlen=MAX_LENGTH, padding='post')

In [ ]:
model = keras.Sequential([
    keras.layers.Embedding(VOCAB_SIZE, 100, input_length=MAX_LENGTH),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

from tf.keras.callbacks import EarlyStopping
monitor = EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=25, verbose=1, mode='max', restore_best_weights=True)

model.fit(padded_train_message, train_label, validation_data=(padded_test_message, test_label), callbacks=[monitor], epochs=1000, verbose=2)


Epoch 1/1000
131/131 - 12s - loss: 0.1669 - accuracy: 0.9390 - val_loss: 0.0755 - val_accuracy: 0.9756 - 12s/epoch - 88ms/step
Epoch 2/1000
131/131 - 15s - loss: 0.0308 - accuracy: 0.9902 - val_loss: 0.0412 - val_accuracy: 0.9849 - 15s/epoch - 115ms/step
Epoch 3/1000
131/131 - 11s - loss: 0.0085 - accuracy: 0.9983 - val_loss: 0.0363 - val_accuracy: 0.9878 - 11s/epoch - 82ms/step
Epoch 4/1000
131/131 - 6s - loss: 0.0034 - accuracy: 0.9998 - val_loss: 0.0396 - val_accuracy: 0.9864 - 6s/epoch - 45ms/step
Epoch 5/1000
131/131 - 10s - loss: 0.0020 - accuracy: 0.9998 - val_loss: 0.0369 - val_accuracy: 0.9871 - 10s/epoch - 74ms/step
Epoch 6/1000
131/131 - 7s - loss: 0.0014 - accuracy: 0.9998 - val_loss: 0.0487 - val_accuracy: 0.9864 - 7s/epoch - 53ms/step
Epoch 7/1000
131/131 - 9s - loss: 6.9382e-04 - accuracy: 0.9998 - val_loss: 0.0425 - val_accuracy: 0.9871 - 9s/epoch - 67ms/step
Epoch 8/1000
131/131 - 7s - loss: 3.0501e-04 - accuracy: 1.0000 - val_loss: 0.0443 - val_accuracy: 0.9864 - 7s/e

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  class_dict = {
      0 : "ham",
      1 : "spam",
      }
  encoded_message = [one_hot(pred_text, VOCAB_SIZE)]
  padded_message = pad_sequences(encoded_message, maxlen=MAX_LENGTH, padding='post')
  prediction = [model.predict(padded_message)[0][0], class_dict[np.round(model.predict(padded_message)[0][0])]]
  print(prediction)
  return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 27ms/step
[0.0009374482, 'ham']
[0.0009374482, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 45ms/step
[0.0009374482, 'ham']
1/1 [==============================] - 0s 29ms/step
[0.5189217, 'spam']
1/1 [==============================] - 0s 30ms/step
[6.260659e-05, 'ham']
1/1 [==============================] - 0s 22ms/step
[0.99578273, 'spam']
1/1 [==============================] - 0s 23ms/step
[0.98986423, 'spam']
1/1 [==============================] - 0s 22ms/step
[0.00028447591, 'ham']
1/1 [==============================] - 0s 26ms/step
[0.0029728862, 'ham']
You passed the challenge. Great job!
